In [4]:
import cv2
import os
import numpy as np


def extract_faces(input_folder, output_folder):
    # Load the Caffe model for face detection using SSD
    modelFile = "../HAAR_Models/res10/res10_300x300_ssd_iter_140000.caffemodel"
    configFile = "../HAAR_Models/res10/deploy.prototxt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

        # Set the confidence threshold for face detection
    confidence_threshold = 0.5

    # Loop through all images in the input folder
    for filename in os.listdir(input_folder):
        try:
            # Load the image and get its dimensions
            image = cv2.imread(os.path.join(input_folder, filename))
            (h, w) = image.shape[:2]

            # Prepare the blob to be fed into the model
            blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

            # Feed the blob into the model to get the detections
            net.setInput(blob)
            detections = net.forward()

            faces_detected = False
            # Loop through the detections and extract faces
            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > confidence_threshold:
                    faces_detected = True
                    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                    (startX, startY, endX, endY) = box.astype("int")

                    # Extract the face and save it in the output folder
                    face = image[startY:endY, startX:endX]
                    output_path = os.path.join(output_folder, f"face_{i}_{filename}")
                    cv2.imwrite(output_path, face)
            if not faces_detected:
                print(f"No faces detected in image: {filename}")
        except Exception as e:
            print(f"Error processing image: {filename}. Error: {e}")

if __name__ == '__main__':
    input_folder = "../dataset/Example/"
    output_folder = "../cropped_faces2/"

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    extract_faces(input_folder, output_folder)

Error processing image: 1.2017-09-09-06-55-51.jpg. Error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'

